# Chapter 40: Bundle Size Optimization

Bundle size directly impacts application performance, user experience, and SEO rankings. Large bundles increase load times, especially on mobile networks, and consume more memory during parsing and execution. This chapter explores strategies to minimize TypeScript application bundles while maintaining functionality and performance.

---

## 40.1 Tree Shaking and Dead Code Elimination

Tree shaking is a form of dead code elimination that removes unused exports from the final bundle. Modern bundlers like Rollup, Webpack, and esbuild perform tree shaking based on ES module static structure.

### How Tree Shaking Works

```typescript
// utils.ts - Source file
export function formatDate(date: Date): string {
  return date.toISOString();
}

export function formatCurrency(amount: number): string {
  return `$${amount.toFixed(2)}`;
}

export function heavyCalculation(data: unknown[]): unknown[] {
  // Complex processing that might not be needed
  return data.map(item => processItem(item));
}

function processItem(item: unknown): unknown {
  return item;
}

// main.ts - Entry point
import { formatDate } from './utils';

console.log(formatDate(new Date()));
// Result: Only formatDate is included in bundle
// formatCurrency, heavyCalculation, and processItem are removed
```

**Requirements for Effective Tree Shaking:**

```json
// tsconfig.json
{
  "compilerOptions": {
    "module": "ESNext",        // ES modules required for tree shaking
    "moduleResolution": "bundler",
    "target": "ES2020",
    "declaration": true,
    "declarationMap": true
  }
}
```

```javascript
// webpack.config.js
module.exports = {
  mode: 'production',
  optimization: {
    usedExports: true,      // Mark unused exports
    sideEffects: false,     // Assume no side effects (enable tree shaking)
    
    // Split chunks for better caching
    splitChunks: {
      chunks: 'all',
      cacheGroups: {
        vendor: {
          test: /[\\/]node_modules[\\/]/,
          name: 'vendors',
          chunks: 'all',
        },
      },
    },
  },
};
```

### Side Effects and Pure Modules

```json
// package.json
{
  "name": "my-library",
  "sideEffects": false,  // Entire package is tree-shakeable
  // OR
  "sideEffects": [
    "*.css",
    "*.scss",
    "./src/polyfills.ts"  // Only these files have side effects
  ]
}
```

**Side Effect Patterns:**

```typescript
// polyfills.ts - Must be marked as side effect
import 'core-js/features/array/flat-map';
import 'regenerator-runtime/runtime';

// utils.ts - Pure module (no side effects)
export const PI = 3.14159;

export function calculateCircumference(radius: number): number {
  return 2 * PI * radius;
}

// This would be tree-shaken if unused
export function calculateArea(radius: number): number {
  return PI * radius * radius;
}
```

### Barrel Files and Tree Shaking

Barrel files (index.ts) can hinder tree shaking if not structured properly:

```typescript
// ❌ Bad: Barrel file that imports everything
// components/index.ts
import { Button } from './Button';
import { Modal } from './Modal';      // Always imported even if unused
import { Table } from './Table';      // Always imported even if unused
import { Chart } from './Chart';      // Heavy dependency always imported

export { Button, Modal, Table, Chart };

// ❌ Bad: Importing from barrel
import { Button } from './components'; // Brings in all components

// ✅ Good: Direct imports for better tree shaking
import { Button } from './components/Button';
import { Modal } from './components/Modal';

// ✅ Good: Barrel with proper structure
// components/index.ts
export { Button } from './Button';
export { Modal } from './Modal';
export { Table } from './Table';
export { Chart } from './Chart';

// But still, import only what you need:
import { Button } from './components/Button'; // Better than './components'
```

---

## 40.2 Analyzing Bundle Composition

Understanding what contributes to bundle size is essential for optimization.

### Bundle Analysis Tools

```bash
# Install webpack-bundle-analyzer
npm install -D webpack-bundle-analyzer

# Build with analyzer
npx webpack --analyze

# OR use as plugin
```

```javascript
// webpack.config.js
const BundleAnalyzerPlugin = require('webpack-bundle-analyzer').BundleAnalyzerPlugin;

module.exports = {
  plugins: [
    new BundleAnalyzerPlugin({
      analyzerMode: 'static',  // Generate HTML report
      openAnalyzer: false,
      reportFilename: 'bundle-report.html',
    }),
  ],
};
```

**Vite Bundle Analysis:**

```bash
npm install -D rollup-plugin-visualizer
```

```typescript
// vite.config.ts
import { defineConfig } from 'vite';
import { visualizer } from 'rollup-plugin-visualizer';

export default defineConfig({
  build: {
    rollupOptions: {
      plugins: [
        visualizer({
          filename: './dist/stats.html',
          open: true,
          gzipSize: true,
          brotliSize: true,
        }),
      ],
    },
  },
});
```

### Programmatic Bundle Analysis

```typescript
// scripts/analyze-bundle.ts
import fs from 'fs';
import path from 'path';
import { gzipSync } from 'zlib';

interface FileStats {
  path: string;
  size: number;
  gzipSize: number;
}

function analyzeDirectory(dir: string): FileStats[] {
  const files: FileStats[] = [];
  
  function walk(currentPath: string) {
    const stats = fs.statSync(currentPath);
    
    if (stats.isDirectory()) {
      fs.readdirSync(currentPath).forEach(file => {
        walk(path.join(currentPath, file));
      });
    } else if (currentPath.endsWith('.js') || currentPath.endsWith('.mjs')) {
      const content = fs.readFileSync(currentPath);
      const gzipSize = gzipSync(content).length;
      
      files.push({
        path: currentPath,
        size: stats.size,
        gzipSize,
      });
    }
  }
  
  walk(dir);
  return files.sort((a, b) => b.size - a.size);
}

const stats = analyzeDirectory('./dist');
console.table(stats.map(s => ({
  file: path.relative('./dist', s.path),
  size: `${(s.size / 1024).toFixed(2)} KB`,
  gzip: `${(s.gzipSize / 1024).toFixed(2)} KB`,
})));
```

### Identifying Large Dependencies

```bash
# Using bundle-phobia-cli
npx bundle-phobia lodash
npx bundle-phobia moment
npx bundle-phobia @mui/material

# Using import-cost extension (VS Code)
# Shows inline size estimates for imports
```

**Size Budget Configuration:**

```json
// package.json
{
  "bundlesize": [
    {
      "path": "./dist/main.*.js",
      "maxSize": "150 KB"
    },
    {
      "path": "./dist/vendor.*.js",
      "maxSize": "250 KB"
    }
  ]
}
```

```javascript
// webpack.config.js
module.exports = {
  performance: {
    hints: 'error',  // or 'warning'
    maxEntrypointSize: 400000,  // 400 KB
    maxAssetSize: 250000,       // 250 KB
    
    // Exclude certain assets from size checks
    assetFilter: function(assetFilename) {
      return assetFilename.endsWith('.js');
    },
  },
};
```

---

## 40.3 Dynamic Imports and Code Splitting

Code splitting breaks the bundle into smaller chunks loaded on demand, reducing initial load time.

### Dynamic Import Syntax

```typescript
// Static import (included in main bundle)
import { HeavyComponent } from './HeavyComponent';

// Dynamic import (loaded on demand)
const HeavyComponent = lazy(() => import('./HeavyComponent'));

// With TypeScript types
interface ChartModule {
  default: React.ComponentType<ChartProps>;
  ChartTypes: typeof ChartTypes;
}

const loadChart = async (): Promise<ChartModule> => {
  const module = await import('./Chart');
  return module;
};

// React lazy loading with TypeScript
import { lazy, Suspense } from 'react';

const AdminDashboard = lazy(() => import('./pages/AdminDashboard'));
const UserProfile = lazy(() => import('./pages/UserProfile'));

function App() {
  return (
    <Suspense fallback={<div>Loading...</div>}>
      <Routes>
        <Route path="/admin" element={<AdminDashboard />} />
        <Route path="/profile" element={<UserProfile />} />
      </Routes>
    </Suspense>
  );
}
```

### Prefetching and Preloading

```typescript
// Prefetch: Load when browser is idle
const AdminModule = lazy(() => import(
  /* webpackPrefetch: true */ 
  /* webpackChunkName: "admin" */
  './AdminModule'
));

// Preload: Load immediately (higher priority)
const CriticalModule = lazy(() => import(
  /* webpackPreload: true */
  './CriticalModule'
));

// Programmatic prefetching
const prefetchRoutes = () => {
  // Prefetch likely next routes
  const likelyRoute = predictNextRoute();
  if (likelyRoute === 'dashboard') {
    import(/* webpackPrefetch: true */ './Dashboard');
  }
};
```

### Route-Based Code Splitting

```typescript
// router.tsx with lazy loading
import { createBrowserRouter } from 'react-router-dom';
import { lazy, Suspense } from 'react';
import { Layout } from './components/Layout'; // Keep layout in main bundle

// Lazy load route components
const HomePage = lazy(() => import('./pages/Home'));
const ProductPage = lazy(() => import('./pages/Product'));
const CartPage = lazy(() => import('./pages/Cart'));
const CheckoutPage = lazy(() => import('./pages/Checkout'));

export const router = createBrowserRouter([
  {
    path: '/',
    element: <Layout />,
    children: [
      { index: true, element: <HomePage /> },
      { 
        path: 'product/:id', 
        element: (
          <Suspense fallback={<ProductSkeleton />}>
            <ProductPage />
          </Suspense>
        )
      },
      { 
        path: 'cart', 
        element: (
          <Suspense fallback={<CartSkeleton />}>
            <CartPage />
          </Suspense>
        )
      },
      { 
        path: 'checkout', 
        element: (
          <Suspense fallback={<CheckoutSkeleton />}>
            <CheckoutPage />
          </Suspense>
        ),
        // Load checkout chunk when hovering over cart
        loader: () => {
          import('./pages/Checkout'); // Trigger prefetch
          return null;
        }
      },
    ],
  },
]);
```

### Vendor Splitting

```javascript
// webpack.config.js
module.exports = {
  optimization: {
    splitChunks: {
      chunks: 'all',
      maxInitialRequests: Infinity,
      minSize: 0,
      cacheGroups: {
        // React vendor chunk
        reactVendor: {
          test: /[\\/]node_modules[\\/](react|react-dom|react-router-dom)[\\/]/,
          name: 'vendor-react',
          chunks: 'all',
        },
        // UI library chunk
        uiVendor: {
          test: /[\\/]node_modules[\\/](@mui|@chakra-ui|antd)[\\/]/,
          name: 'vendor-ui',
          chunks: 'all',
        },
        // Utility libraries
        utilsVendor: {
          test: /[\\/]node_modules[\\/](lodash|moment|dayjs)[\\/]/,
          name: 'vendor-utils',
          chunks: 'all',
        },
        // All other node_modules
        defaultVendors: {
          test: /[\\/]node_modules[\\/]/,
          priority: -10,
          reuseExistingChunk: true,
        },
      },
    },
  },
};
```

---

## 40.4 Minification Strategies

Minification reduces code size by removing whitespace, shortening variable names, and optimizing syntax.

### Terser Configuration

```javascript
// webpack.config.js
const TerserPlugin = require('terser-webpack-plugin');

module.exports = {
  optimization: {
    minimize: true,
    minimizer: [
      new TerserPlugin({
        terserOptions: {
          parse: {
            ecma: 2020,
          },
          compress: {
            ecma: 5,
            warnings: false,
            comparisons: false,
            inline: 2,
            drop_console: true,      // Remove console statements
            drop_debugger: true,     // Remove debugger statements
            pure_funcs: ['console.log', 'console.info'], // Remove specific calls
            passes: 2,               // Multiple optimization passes
          },
          mangle: {
            safari10: true,          // Workaround Safari 10 bugs
            properties: {
              regex: /^_/,          // Mangle private properties
            },
          },
          format: {
            comments: false,         // Remove comments
            ascii_only: true,        // Escape Unicode for older browsers
          },
        },
        parallel: true,              // Use multiple cores
        extractComments: false,      // Don't extract licenses to separate file
      }),
    ],
  },
};
```

### esbuild Minification (Faster)

```javascript
// esbuild.config.js
const esbuild = require('esbuild');

esbuild.build({
  entryPoints: ['src/index.ts'],
  bundle: true,
  minify: true,           // Enable minification
  minifyWhitespace: true,
  minifyIdentifiers: true,
  minifySyntax: true,
  treeShaking: true,
  target: 'es2020',
  outdir: 'dist',
});
```

### Compression Plugins

```javascript
// webpack.config.js
const CompressionPlugin = require('compression-webpack-plugin');

module.exports = {
  plugins: [
    // Gzip compression
    new CompressionPlugin({
      algorithm: 'gzip',
      test: /\.(js|css|html|svg)$/,
      threshold: 8192,        // Only compress > 8KB
      minRatio: 0.8,
    }),
    
    // Brotli compression (better than gzip)
    new CompressionPlugin({
      algorithm: 'brotliCompress',
      test: /\.(js|css|html|svg)$/,
      compressionOptions: {
        params: {
          [require('zlib').constants.BROTLI_PARAM_QUALITY]: 11, // Max compression
        },
      },
      threshold: 8192,
      minRatio: 0.8,
      filename: '[path][base].br',
    }),
  ],
};
```

---

## 40.5 Import Optimization

How you import dependencies significantly impacts bundle size.

### Selective Imports

```typescript
// ❌ Bad: Import entire library
import _ from 'lodash';                    // ~70KB
import * as moment from 'moment';          // ~290KB with locales
import { Button, Modal } from '@mui/material'; // Brings in all Material UI

// ✅ Good: Import only what you need
import debounce from 'lodash/debounce';     // ~2KB
import isEqual from 'lodash/isEqual';      // ~5KB
import dayjs from 'dayjs';                 // ~6KB (moment alternative)
import Button from '@mui/material/Button'; // Only Button + dependencies
import Modal from '@mui/material/Modal';

// ✅ Good: Use tree-shakeable alternatives
import { debounce, isEqual } from 'es-toolkit'; // Modern, tree-shakeable
import { format } from 'date-fns';              // Tree-shakeable dates
```

### Babel Plugin for Lodash

```bash
npm install -D babel-plugin-lodash @babel/plugin-transform-runtime
```

```json
// .babelrc
{
  "plugins": [
    "lodash",  // Transforms `import _ from 'lodash'` to cherry-picked imports
    ["@babel/plugin-transform-runtime", { "useESModules": true }]
  ]
}
```

### Module Resolution Optimization

```json
// tsconfig.json
{
  "compilerOptions": {
    "moduleResolution": "bundler",  // Modern resolution (faster)
    "paths": {
      // Alias heavy imports to lighter alternatives
      "moment": ["./node_modules/dayjs"],
      "lodash": ["./node_modules/lodash-es"]
    }
  }
}
```

### Date Library Comparison

```typescript
// Moment (legacy, large)
import moment from 'moment';           // 290KB + locales
moment().format('YYYY-MM-DD');

// Dayjs (lightweight, moment-compatible)
import dayjs from 'dayjs';             // 6KB
dayjs().format('YYYY-MM-DD');

// Date-fns (tree-shakeable)
import { format } from 'date-fns';   // Import only what you use (~5KB for format)
format(new Date(), 'yyyy-MM-dd');

// Native Intl (no bundle cost)
new Intl.DateTimeFormat('en-US').format(new Date());
```

---

## 40.6 Third-Party Library Size Management

Evaluating and optimizing external dependencies is crucial for bundle size.

### Bundle Phobia Analysis

```bash
# Check size impact before installing
npx bundle-phobia react-query
npx bundle-phobia @tanstack/react-query

# Compare alternatives
npx bundle-phobia lodash           # 69.9KB
npx bundle-phobia lodash-es        # 89.3KB (but tree-shakeable)
npx bundle-phobia ramda            # 78.4KB
npx bundle-phobia radash           # 9.8KB (modern alternative)
```

### Dependency Replacement Strategy

```typescript
// utils/bundle-optimization.ts

// Replace heavy libraries with lighter alternatives
export { debounce, throttle } from 'es-toolkit';  // Instead of lodash
export { clsx, type ClassValue } from 'clsx';     // Instead of classnames (smaller)

// Custom lightweight implementations
export const isObject = (value: unknown): value is Record<string, unknown> =>
  typeof value === 'object' && value !== null && !Array.isArray(value);

export const pick = <T extends object, K extends keyof T>(
  obj: T,
  keys: K[]
): Pick<T, K> => {
  const result = {} as Pick<T, K>;
  keys.forEach(key => {
    if (key in obj) {
      result[key] = obj[key];
    }
  });
  return result;
};
```

### External Dependencies (CDN)

```html
<!-- index.html - Load heavy libraries from CDN -->
<script src="https://cdn.jsdelivr.net/npm/react@18/umd/react.production.min.js" crossorigin></script>
<script src="https://cdn.jsdelivr.net/npm/react-dom@18/umd/react-dom.production.min.js" crossorigin></script>
<script src="https://cdn.jsdelivr.net/npm/lodash@4/lodash.min.js" crossorigin></script>

<script>
  // Make globals available as modules
  window.process = { env: { NODE_ENV: 'production' } };
</script>
```

```javascript
// webpack.config.js
module.exports = {
  externals: {
    react: 'React',
    'react-dom': 'ReactDOM',
    lodash: '_',
  },
};

// TypeScript declarations for externals
// types/externals.d.ts
declare const React: typeof import('react');
declare const ReactDOM: typeof import('react-dom');
declare const _: typeof import('lodash');
```

---

## 40.7 Compression and Delivery Optimization

Network transfer size is as important as bundle size.

### Build-Time Compression

```javascript
// vite.config.ts
import { defineConfig } from 'vite';
import gzipPlugin from 'rollup-plugin-gzip';
import { brotliCompressSync } from 'zlib';

export default defineConfig({
  build: {
    rollupOptions: {
      plugins: [
        gzipPlugin({
          filter: /\.(js|mjs|json|css|html)$/,
          fileName: '.gz',
        }),
        {
          name: 'brotli',
          writeBundle: {
            order: 'post',
            handler({ dir }) {
              // Custom brotli compression
              const files = fs.readdirSync(dir);
              files.forEach(file => {
                if (file.match(/\.(js|css)$/)) {
                  const content = fs.readFileSync(path.join(dir, file));
                  const compressed = brotliCompressSync(content, {
                    params: {
                      [constants.BROTLI_PARAM_QUALITY]: 11,
                    },
                  });
                  fs.writeFileSync(path.join(dir, `${file}.br`), compressed);
                }
              });
            },
          },
        },
      ],
    },
  },
});
```

### Server Configuration

**Nginx:**
```nginx
# nginx.conf
gzip on;
gzip_vary on;
gzip_proxied any;
gzip_comp_level 6;
gzip_types text/plain text/css text/xml application/json application/javascript application/rss+xml application/atom+xml image/svg+xml;

# Brotli (requires ngx_brotli module)
brotli on;
brotli_comp_level 11;
brotli_types text/plain text/css application/javascript application/json image/svg+xml;
```

**Express:**
```typescript
// server.ts
import express from 'express';
import compression from 'compression';

const app = express();

// Enable gzip compression
app.use(compression({
  level: 6,
  filter: (req, res) => {
    if (req.headers['x-no-compression']) {
      return false;
    }
    return compression.filter(req, res);
  },
}));

// Serve pre-compressed files
app.get('*.js', (req, res, next) => {
  if (req.header('Accept-Encoding').includes('br')) {
    req.url += '.br';
    res.set('Content-Encoding', 'br');
    res.set('Content-Type', 'application/javascript');
  } else if (req.header('Accept-Encoding').includes('gzip')) {
    req.url += '.gz';
    res.set('Content-Encoding', 'gzip');
    res.set('Content-Type', 'application/javascript');
  }
  next();
});

app.use(express.static('dist'));
```

---

## 40.8 Runtime Performance vs Bundle Size Trade-offs

Sometimes larger bundles can improve runtime performance through optimizations.

### Pre-compiled vs Runtime Compilation

```typescript
// Smaller bundle, but runtime overhead
import { compileTemplate } from 'template-compiler';
const template = compileTemplate(source); // Runtime compilation

// Larger bundle, but faster execution
import { precompiledTemplate } from './templates'; // Build-time compilation
```

### Inline vs External Source Maps

```javascript
// webpack.config.js
module.exports = {
  devtool: 'source-map',        // External file (production)
  // devtool: 'eval-source-map', // Inline (development, faster rebuild)
  // devtool: false,             // No source maps (smallest, hardest to debug)
  
  optimization: {
    // Runtime chunk separates webpack runtime for better caching
    runtimeChunk: 'single',
    
    // Module ids affect caching
    moduleIds: 'deterministic', // Short numeric ids
    // moduleIds: 'named',      // Readable but larger
  },
};
```

### Polyfill Strategy

```typescript
// ❌ Bad: Import all polyfills
import 'core-js';  // Huge bundle impact

// ✅ Good: Feature detection and conditional loading
async function loadPolyfills() {
  const polyfills = [];
  
  if (!('IntersectionObserver' in window)) {
    polyfills.push(import('intersection-observer'));
  }
  
  if (!('fetch' in window)) {
    polyfills.push(import('whatwg-fetch'));
  }
  
  await Promise.all(polyfills);
}

// ✅ Good: Polyfill.io service (only sends needed polyfills)
// <script src="https://polyfill.io/v3/polyfill.min.js?features=fetch,IntersectionObserver"></script>
```

---

## 40.9 Chapter Summary and Exercises

### Chapter Summary

This chapter covered bundle size optimization for TypeScript applications:

**Key Concepts:**

1. **Tree Shaking**: ES modules enable dead code elimination. Ensure `sideEffects: false` in package.json, use ES modules (`"module": "ESNext"`), and avoid importing from barrel files when possible.

2. **Bundle Analysis**: Tools like `webpack-bundle-analyzer` and `rollup-plugin-visualizer` identify size contributors. Set size budgets in webpack or CI pipelines to prevent regression.

3. **Code Splitting**: Dynamic imports (`import()`) load code on demand. Implement route-based splitting, lazy loading for heavy components, and vendor chunk splitting for better caching.

4. **Minification**: Terser and esbuild remove whitespace and shorten names. Configure `drop_console: true` to remove debug code. Brotli compression achieves 20-30% better ratios than gzip.

5. **Import Optimization**: Import specific functions (`import { debounce } from 'lodash'`) rather than entire libraries (`import _ from 'lodash'`). Replace heavy libraries (moment.js → dayjs/date-fns).

6. **Library Management**: Evaluate dependencies with `bundle-phobia` before installing. Consider lighter alternatives (radash vs lodash). Use externals for CDN-hosted libraries.

7. **Compression**: Enable gzip and brotli on servers. Pre-compress assets at build time for static hosting. Configure appropriate compression levels (6-9 for gzip, 9-11 for brotli).

8. **Trade-offs**: Balance bundle size against runtime performance. Sometimes larger pre-computed tables are faster than runtime calculations. Source maps increase size but are essential for debugging.

### Practical Exercises

**Exercise 1: Bundle Analysis**

Set up bundle analysis for a project:
- Configure `webpack-bundle-analyzer` or `rollup-plugin-visualizer`
- Identify the largest dependencies in the bundle
- Create a report showing which imports contribute most to size
- Set up size budgets that fail the build if exceeded

**Exercise 2: Tree Shaking Optimization**

Take an existing project and optimize tree shaking:
- Convert CommonJS imports to ES modules
- Configure `sideEffects: false` in package.json
- Replace `import * as _ from 'lodash'` with specific imports
- Measure bundle size before and after optimizations

**Exercise 3: Code Splitting Implementation**

Implement code splitting in a React application:
- Convert static imports to dynamic imports for routes
- Implement React.lazy() and Suspense for code splitting
- Configure webpack to split vendor chunks (React, UI library, utilities separately)
- Measure initial load time improvement

**Exercise 4: Library Replacement**

Replace heavy dependencies with lighter alternatives:
- Replace moment.js with date-fns or dayjs
- Replace lodash with es-toolkit or radash
- Replace heavy UI component library with lighter alternative
- Measure bundle size reduction and verify functionality remains intact

**Exercise 5: Compression Setup**

Implement full compression pipeline:
- Configure build-time gzip and brotli generation
- Set up Nginx or Express to serve pre-compressed files
- Implement content negotiation (Accept-Encoding header handling)
- Measure transfer size reduction with browser DevTools Network panel

**Exercise 6: Performance Budgeting**

Implement performance budgets in CI:
- Configure `bundlesize` or `size-limit` in package.json
- Set thresholds for JavaScript, CSS, and image assets
- Integrate with GitHub Actions to comment on PRs with size changes
- Block merges that increase bundle size beyond thresholds

### Additional Resources

- **Webpack Bundle Analyzer**: https://github.com/webpack-contrib/webpack-bundle-analyzer
- **Bundle Phobia**: https://bundlephobia.com/
- **Import Cost VS Code Extension**: Shows import sizes inline
- **Web.dev Bundle Optimization**: https://web.dev/reduce-javascript-payloads-with-code-splitting/
- **Lighthouse**: Built-in Chrome tool for performance auditing

---

## Coming Up Next: Chapter 41 - Popular TypeScript Libraries

In the next chapter, we will explore popular TypeScript libraries that enhance development:

- Zod - Runtime type validation with TypeScript inference
- TypeBox - JSON Schema type builder
- io-ts - Runtime type validation for functional programming
- ts-pattern - Exhaustive pattern matching
- Effect - Functional programming toolkit
- Prisma - Type-safe database toolkit
- tRPC - End-to-end typesafe APIs
- Zustand - Lightweight state management

Understanding these libraries extends TypeScript's capabilities beyond the compiler, providing runtime validation, functional patterns, and type-safe integrations with databases and APIs.